In [ ]:
import plotly.express as px
import json
import pandas as pd
import dash
import os
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pydeck as pdk

In [ ]:
#load the data from the sentiment_full_updated.xlsx file and load it into a dataframe

series_type = type(pd.Series([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]))

dtype_dict = {
    'date': str,
    'Hiker trail name': str,
    'Hiker Journal Link': str,
    'Journal Story': str,
    'Start location': str,
    'Destination': str,
    'Today Miles': str,
    'Latitude': str,
    'Longitude': str,
    'State': str,
    'Total Shelters': str,
    'Occurrence': str,
    'year': str,
    'month': str,
    'label': str,
    # pandas serial object for type of emotion score
    'Unnamed: 0': str


}


df = pd.read_excel('../sentiment_full_updated_with_coords.xlsx', sheet_name='Sheet1', dtype=dtype_dict)

print(f'length of df: {len(df)}')

In [ ]:
print(f'columsn in df are {df.columns}')
print(f'number of columns in df is {len(df.columns)}')



In [ ]:
# clean the data frame by dropping the column called 'Unnamed: 0'
df = df.drop(columns=['Unnamed: 0'])
print(f'columsn in df are {df.columns}')
print(f'number of columns in df is {len(df.columns)}')

In [ ]:
#print the data type of each column
print(df.dtypes)
print(df.head()["Hiker trail name"].values[0])

In [ ]:
# clean the data frame by dropping the specifiying the data type of each column


# print out a random sample of each of the columns alongside its data type
for col in df.columns:
    print(f'column {col} has data type {type(df[col].values[0])}')
    temp_sample = df[col].sample(1)
    print(temp_sample.values[0])
    print('')

# cast all elemnts in Start location to string
df['Start location'] = df['Start location'].astype(str)

In [ ]:
# grab a row called test_row_with_nan_coords from this link https://www.trailjournals.com//journal/entry/518909
sample = df[df['Hiker Journal Link'] == 'https://www.trailjournals.com//journal/entry/518909']

# print out the sample with the format column names: values
for column in sample.columns:
    print("-------------------------")
    print(column + ': ' + str(sample[column].values[0]))
    print("")
    print(f'type of {column} is {type(sample[column].values[0])}')

    

In [ ]:
# print(df.head()['Emotion_scores'].values[0][3])
# print(df.head()['Emotion_scores'].values[0])
# new_emo = eval(df.head()['Emotion_scores'].values[0])
# print(type(new_emo))

emotion_nan_counter = 0
# loop over all the values in the Emotion_scores column and convert them to dictionaries
for i in range(len(df['Emotion_scores'].values)):
    super_value = df['Emotion_scores'].values[i]
    # print(f'type of super_value is {type(super_value)}')
    # print(f'super_value is {super_value}')
    if type(super_value) != type("string"):
        emotion_nan_counter += 1
        continue
    value = super_value[0]
    # print(value)
    # print(f'type of value is {type(value)}')

print(f'number of nan values in Emotion_scores is {emotion_nan_counter}')
print(f'number of non-nan values in Emotion_scores is {len(df["Emotion_scores"].values) - emotion_nan_counter}')
print(f'total number of values in Emotion_scores is {len(df["Emotion_scores"].values)}')

In [ ]:
# drop any rows that have do not have objects of type string in the Emotion_scores column in their df['Emotion_scores'].values[i]
df = df[df['Emotion_scores'].apply(lambda x: type(x) == type("string"))]

print(f'number of rows in df is {len(df)}')
# using eval to convert the string to a dictionary do that on the entire colum of Emotion_scores

df['Emotion_scores'] = df['Emotion_scores'].apply(lambda x: eval(x))



# print the type of the first element in the Emotion_scores column
print(type(df['Emotion_scores'].values[0]))





In [ ]:
new_thing = df.head(1)['Emotion_scores'][0][0]['score']
print(type(new_thing))
print(new_thing)

In [ ]:
sample_new_lat = sample['Latitude'].values[0]
print(sample_new_lat)
print(type(sample_new_lat))

sample = df[df['Hiker Journal Link'] == 'https://www.trailjournals.com//journal/entry/519033']
sample_new_lat = sample['Latitude'].values[0]
print(sample_new_lat)
print(type(sample_new_lat))


In [ ]:
# print the values of the latitude column in the df if the type of the value is not a float
count = 0
for i in range(len(df['Latitude'].values)):
    value = df['Latitude'].values[i]
    if type(value) != type('string'):
        count += 1
        # print(value)
        # print(type(value))
print(f'number of float values in Latitude is {count}')
print(f'number of non-float values in Latitude is {len(df["Latitude"].values) - count}')
print(f'total number of values in Latitude is {len(df["Latitude"].values)}')

In [ ]:
# loop over every row in the df and print the value of the lat and long columns
import math

woah_count = 0
not_woah_count = 0
invalid_coords = []
for i in range(len(df['Latitude'].values)):
    lat = df['Latitude'].values[i]
    long = df['Longitude'].values[i]
    if type(lat) != type('string'):
        if math.isnan(lat) or math.isnan(long):
            woah_count += 1
            
            invalid_coords.append(i)
            continue
            # print("woah")
        not_woah_count += 1
    # print(f'lat is {lat} and long is {long}')
    # print(f'type of lat is {type(lat)} and type of long is {type(long)}')
    # print('')

print(f'number of nan values in Latitude is {woah_count}')
print(f'not nan values in Latitude is {not_woah_count}')
print(f'len of invalid_coords is {len(invalid_coords)}')
invalid_coords.sort(reverse=True)

In [ ]:
from thefuzz import fuzz
from thefuzz import process

exclude_words = [
    "shelter",
    "shelters",
    "SHELTER",
    "sheltered",
    "Shelter",
    "Shelters",
    "SHELTERED",
    "SHELTERS",
]

def custom_scorer(s1, s2):
    s1_tokens = [token for token in s1.split() if token not in exclude_words]
    s2_tokens = [token for token in s2.split() if token not in exclude_words]
    return fuzz.token_set_ratio(s1_tokens, s2_tokens)

In [ ]:
# open the weather data file and load it into a dictionary
with open('../weather_data.json') as json_file:
    weather_data = json.load(json_file)
print(f'type of weather_data is {type(weather_data)}')
print(f'len of weather_data is {len(weather_data)}')

In [ ]:
found_in_weather_data = 0
bad_weather_data = 0
not_found = 0
# get first key in weather_data
# first_key = list(weather_data.keys())[0]
# print(f'first key in weather_data is {first_key}')
# weather_data_dict = weather_data[first_key]
# print(f'weather_data_dict is {weather_data_dict}')
# print(f'type of weather_data_dict is {type(weather_data_dict)}')
# print(f'leys in weather_data_dict are {weather_data_dict.keys()}')

for idx in invalid_coords:
    # grab 3 values from the df, the journal link, the latitude, and the longitude
    journal_link = df['Hiker Journal Link'].values[idx]
    lat = df['Latitude'].values[idx]
    long = df['Longitude'].values[idx]
    if journal_link in weather_data:
        curr_dict = weather_data[journal_link]
        curr_dict_keys = curr_dict.keys()
        if 'cod' in curr_dict_keys:
            bad_weather_data += 1
        else:
            # replace the "Latitude" and "Longitude" in the df with the values in the curr dict from keys with "lat" and "lon" keys
            df['Latitude'].values[idx] = str(curr_dict['lat'])
            df['Longitude'].values[idx] = str(curr_dict['lon'])
            # print(f'found new lat {curr_dict["lat"]} in weather data')
            # print(f'type of "Latitude" is {type(df["Latitude"].values[idx])}')
            found_in_weather_data += 1
    else:
        not_found += 1


print(f'amount found in weather data is {found_in_weather_data}')
print(f'searched every invalid coord: {found_in_weather_data + bad_weather_data + not_found == len(invalid_coords)}')
print(f'difference between invalid_coords and found_in_weather_data is {len(invalid_coords) - (found_in_weather_data + bad_weather_data + not_found)}')


In [ ]:
# drop all the rows that have values in Latitude that are not type string
len_before_drop = len(df)
df = df[df['Latitude'].apply(lambda x: type(x) == type("string"))]
len_after_drop = len(df)
print(f'len_before_drop is {len_before_drop}')
print(f'len_after_drop is {len_after_drop}')
print(f'difference between len_before_drop and len_after_drop is {len_before_drop - len_after_drop}')

In [ ]:
print(f'amount added to the df is {found_in_weather_data}')
print(f'amount not found in weather data is {woah_count - found_in_weather_data}')

In [ ]:
# print the unique values in the year column
print(f'unique values in year column are {df["year"].unique()}')
len_before_drop_year = len(df)
df = df[df['year'].apply(lambda x: type(x) == type("string"))]
len_after_drop_year = len(df)
df = df[df['year'].apply(lambda x: len(x) == len("2016"))]
len_after_drop_bad_year = len(df)
print(f'len_before_drop_year is {len_before_drop_year}')
print(f'len_after_drop_year is {len_after_drop_year}')
print(f'difference between len_before_drop_year and len_after_drop_year is {len_before_drop_year - len_after_drop_year}')
print(f'amount of bad year values is {len_after_drop_year - len_after_drop_bad_year}')
print(f'unique values in year column after drop are {df["year"].unique()}')

In [84]:
print(f'unique values in month column {df["month"].unique()}')

unique values in month column [ 1  2  3  4  5  6  7  8  9 10 11 12]


In [85]:
#helper methods
def count_entries_within_radius(
    target_lon, target_lat, radius_miles, currYear, currMonth
):
    # Convert miles to degrees (roughly, considering Earth's radius)
    degrees_per_mile = 1 / 69  # Approximately
    radius_deg = radius_miles * degrees_per_mile

    # Calculate latitude and longitude bounds for the square around the target point
    min_lon = target_lon - radius_deg
    max_lon = target_lon + radius_deg
    min_lat = target_lat - radius_deg
    max_lat = target_lat + radius_deg

    # Filter the DataFrame to get entries within the specified latitude and longitude bounds
    filtered_df = df[
        (df["lon"] >= min_lon)
        & (df["lon"] <= max_lon)
        & (df["lat"] >= min_lat)
        & (df["lat"] <= max_lat)
        & (df["year"] == currYear)
        & (df["month"] == currMonth)
    ]

    # Get the count of entries within the radius
    count_within_radius = len(filtered_df)

    return count_within_radius


def monthToNum(shortMonth):
    return {
        "Jan": 1,
        "Feb": 2,
        "Mar": 3,
        "Apr": 4,
        "May": 5,
        "Jun": 6,
        "Jul": 7,
        "Aug": 8,
        "Sep": 9,
        "Oct": 10,
        "Nov": 11,
        "Dec": 12,
    }[shortMonth]


def numTofullMonthName(num):
    return {
        1: "January",
        2: "Febuary",
        3: "March",
        4: "April",
        5: "May",
        6: "June",
        7: "July",
        8: "Augest",
        9: "September",
        10: "October",
        11: "November",
        12: "December",
    }[num]

In [86]:
# cast the year column to type int
df['year'] = df['year'].astype(int)
# cast the month column to type int
df['month'] = df['month'].astype(int)

# cast the lat and long columns to type float
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

In [87]:
# print the count of the number of entries in the df per year
print(df['year'].value_counts())

year
2016    3024
2017    2730
2019    2244
2022    2210
2021    1619
2018    1612
2020     531
2023     466
Name: count, dtype: int64


In [88]:
# use plotly and the us-terrain map box to plot all of the data just on a graph so we can see where all the data points are
# Plotting the map

# filter df for the year 2017
filtered_df = df[df['year'] == 2017]
fig = px.scatter_geo(filtered_df, 
                     lat='Latitude', 
                     lon='Longitude',
                     scope='usa',  # Set the map scope to USA
                     title='Points on Map of USA',
                     center=dict(lat=41.90722, lon=-70.0369))


# show the fig
fig.show()

In [89]:
# grab the emotion scores for the first row in the df
emotion_scores = df['Emotion_scores'].values[0]
# pretty prtnt the emotion scores
print(json.dumps(emotion_scores, indent=4, sort_keys=True))

[
    {
        "label": "anger",
        "score": 0.004779126960784197
    },
    {
        "label": "disgust",
        "score": 0.004042619839310646
    },
    {
        "label": "fear",
        "score": 0.06184910982847214
    },
    {
        "label": "joy",
        "score": 0.077281653881073
    },
    {
        "label": "neutral",
        "score": 0.5417671799659729
    },
    {
        "label": "sadness",
        "score": 0.12899380922317505
    },
    {
        "label": "surprise",
        "score": 0.1812864989042282
    }
]


In [90]:
# print the columns of the df
print(df.columns)

# print the unique values in the label column
print(df['label'].unique())

Index(['date', 'Hiker trail name', 'Hiker Journal Link', 'Journal Story',
       'Start location', 'Destination', 'Today Miles', 'Latitude', 'Longitude',
       'State', 'Total Shelters', 'Occurrence', 'year', 'month', 'label',
       'Emotion_scores', 'label_int', 'color'],
      dtype='object')
['neutral' 'joy' 'sadness' 'surprise' 'fear' 'anger' 'disgust']


In [91]:
def labelToInt(label):
    return {
        "surprise": 2,
        "joy": 1,
        "neutral": 0,
        "sadness": -1,
        "anger": -2,
        "disgust": -3,
        "fear": -4,

    }[label]

In [92]:
# add a new column to the df called label_int that is the label column but with the values converted to ints
df['label_int'] = df['label'].apply(lambda x: labelToInt(x))

# print out the counts of the label_int column
print(df['label_int'].value_counts())

label_int
 0    6823
 1    2712
-4    1584
-1    1471
 2    1378
-3     375
-2      93
Name: count, dtype: int64


In [ ]:
fig_2=px.density_mapbox(
        df,
        lat="Latitude",
        lon="Longitude",
        z="label_int",
        radius=2,  # Adjust the radius as needed
        center=dict(lat=41.90722, lon=-70.0369),  # Center of the United States
        zoom=4,  # Adjust the zoom level as needed
        mapbox_style="stamen-terrain",  # You can choose a different map style
        title=f"Temperature Heatmap for everything",
        color_continuous_scale="Inferno",  # Adjust the color scale
        range_color=[-4, 2],  # Set the color scale range
        hover_data={
            "Latitude": False,
            "Longitude": False,
            "label_int": True,
        },  # Include temperature in hover data
    )

fig_2.show()

In [110]:
avg_df = df.groupby(['Latitude', 'Longitude'], as_index=False)['label_int'].mean()

# drop all the rows that have label_int == 0
avg_df = avg_df[avg_df['label_int'] != 0]

# Plotting the density map
fig_3 = px.density_mapbox(
    avg_df,  # Use the grouped DataFrame
    lat="Latitude",
    lon="Longitude",
    z="label_int",
    radius=5,
    center=dict(lat=avg_df['Latitude'].mean(), lon=avg_df['Longitude'].mean()),
    zoom=4,
    mapbox_style="stamen-terrain",
    title="Average Emotion on the Trail over all time from 2016 to 2023",
    color_continuous_scale="Inferno",
    range_color=[avg_df['label_int'].min(), avg_df['label_int'].max()],
    hover_data={
        "Latitude": False,
        "Longitude": False,
        "label_int": True,
    }
)

# edit the key in the layout of fig_3 to have it be labled as sentiment value
fig_3.layout.coloraxis.colorbar.title = "Emotional Value"

'''
"sadness": -1,
        "anger": -2,
        "disgust": -3,
        "fear": -4,

'''

tick_values = [2,1,0,-1,-2,-3]  # Define your tick values
tick_labels = ["surprise", "joy", "neutral", "sadness", "disgust", "fear"]  # Define your custom tick labels

fig_3.update_layout(coloraxis_colorbar=dict(tickvals=tick_values, ticktext=tick_labels))



fig_3.update_layout(width=1200, height=800)
fig_3.show()

# stop showing fig_3

In [ ]:
def convertIntToColor(num):
    # make a map that maps ints to colors, have 2 be a deep red, have -4 be a deep blue and have 0 be white, be a gradient the rest inbetween
    return {
        2: '[255, 0, 0]',
        1: "[255, 128, 0]",
        0: '[255,255, 255]',
        -1: '[255, 255, 0]',
        -2: '[128, 255, 0]',
        -3: '[0, 255, 0]',
        -4: '[0, 255, 128]',
    }[num]

# do the same but the colors are black to white
def convertIntToColor2(num):
    # make a map that maps ints to colors, have 2 be a deep red, have -4 be a deep blue and have 0 be white, be a gradient the rest inbetween
    return {
        2: [0, 0, 0, 0],
        1: [51, 51, 51, 0],
        0: [255,255, 255, 0],
        -1: [204, 204, 204, 0],
        -2: [153, 153, 153, 0],
        -3: [102, 102, 102, 0],
        -4: [51, 51, 51, 0],
    }[num]

df['color'] = df['label_int'].apply(convertIntToColor2)


In [ ]:
# full_layer = pdk.Layer(
# "GridLayer",
# data=df,
# pickable=True,
# extruded=True,
# cellSize=20000,
# get_color='color',
# elevationScale=400,
# get_position=["Longitude", "Latitude"],
# get_fill_color=['color'],
# # get_elevation="density",
# )

In [ ]:
# full_view_state = pdk.ViewState(latitude=df["Latitude"].mean(), longitude=df["Longitude"].mean(), zoom=4, pitch=45)
# # Create a deck
# full_deck = pdk.Deck(layers=[full_layer], initial_view_state=full_view_state,  tooltip={
#         'html': '<b>Number of Hikers in a 5-Mile Radius:</b>',
#         'style': {
#             'color': 'grey'
#         }
#     })

# full_deck.to_html("oct_5.html")

In [ ]:
# print the dimensions and the properties of the df
print(df.shape)

df_dict = df.to_dict(orient='records')
print(f'len of df is {len(df)}')
print(f'len of df_dict is {df_dict}')

#print the keys in the df dict

In [ ]:
print(f'type of df_dict is {type(df_dict)}')

# print first item in df_dict
print(df_dict[0])



In [ ]:
# # save the df_dict to a json file
# with open('df_dict.json', 'w') as fp:
#     json.dump(df_dict, fp)

In [ ]:
# open the df_dict dictionary from the json file
with open('df_dict.json') as json_file:
    json_df_dict = json.load(json_file)

print(f'type of json_df_dict is {type(json_df_dict)}')
print(f'len of json_df_dict is {len(json_df_dict)}')

In [ ]:
with open('../inputData/shelter_data.json') as json_file:
    shelter_data = json.load(json_file)
print(len(shelter_data))

In [32]:
#helper method

def count_of_emotions_in_radius(
    target_lon, target_lat, radius_miles
):
    # Convert miles to degrees (roughly, considering Earth's radius)
    degrees_per_mile = 1 / 69  # Approximately
    radius_deg = radius_miles * degrees_per_mile

    # Calculate latitude and longitude bounds for the square around the target point
    min_lon = target_lon - radius_deg
    max_lon = target_lon + radius_deg
    min_lat = target_lat - radius_deg
    max_lat = target_lat + radius_deg

    emotions = ["surprise", "joy", "sadness", "anger", "disgust", "fear"]
    emo_count = {}
    for emotion in emotions:
        # Filter the DataFrame to get entries within the specified latitude and longitude bounds
        filtered_df = df[
            (df["Longitude"] >= min_lon)
            & (df["Longitude"] <= max_lon)
            & (df["Latitude"] >= min_lat)
            & (df["Latitude"] <= max_lat)
            & (df["label"] == emotion)
        ]

        # Get the count of entries within the radius
        count_within_radius = len(filtered_df)

        emo_count[emotion] = count_within_radius
    
    # sum all the values in the emo_count dictionary
    total = 0
    for key in emo_count.keys():
        total += emo_count[key]
    emo_count['total'] = total

    return emo_count

In [33]:
count_of_joy_per_shelter = {}
count_of_sad_per_shelter = {}
count_of_disgust_per_shelter = {}
count_of_anger_per_shelter = {}
count_of_surprise_per_shelter = {}
count_of_fear_per_shelter = {}
# do we want to count the number of times a emotion is in the range of a shelter or the percentage of the time that an emotion is in the range of a shelter

count_of_emotion_dictionary = {"joy": count_of_joy_per_shelter, "sadness": count_of_sad_per_shelter, "disgust": count_of_disgust_per_shelter, "anger": count_of_anger_per_shelter, "surprise": count_of_surprise_per_shelter, "fear": count_of_fear_per_shelter}
# normalize it and do the percentage 
totalTotal = 0
totalCount = 0
totalArr = []

amount_of_entries_at_shelter = {}
for shelter in shelter_data:
   
    # grab the lat and long of the shelter
    
    lat = shelter_data[shelter]["cordinates"]['latitude']
    long = shelter_data[shelter]["cordinates"]['longitude']

    # print(f"shelter is {shelter}, lat is {lat}, long is {long}")

    emo_count = count_of_emotions_in_radius(long, lat, 5)
    amount_of_entries_at_shelter[shelter] = emo_count['total']
    for key in emo_count.keys():
        if key == 'total':
            continue
        count_of_emotion_dictionary[key][shelter] = round(float(emo_count[key] / emo_count['total']) * 100, 2)
    totalTotal += emo_count['total']
    totalCount += 1
    totalArr.append(emo_count['total'])
print(f'mean total is {totalTotal / totalCount}')
# find the median of the totalArr
totalArr.sort()
print(f'median total is {totalArr[len(totalArr) // 2]}')

top_10_for_each_emotion = {}
for key in count_of_emotion_dictionary.keys():
    
    curr_dict = count_of_emotion_dictionary[key]

    # find the 10 keys with the highest values in the curr_dict
    sorted_keys = sorted(curr_dict, key=curr_dict.get, reverse=True)

    final_sorted_keys = []
    curr_added_count = 1
    currIdx = 0
    while curr_added_count <= 10 and currIdx < len(sorted_keys) and len(final_sorted_keys) < 10:
        if amount_of_entries_at_shelter[sorted_keys[currIdx]] > 10:
            final_sorted_keys.append(sorted_keys[currIdx])
            curr_added_count += 1
        currIdx += 1
    top_10_for_each_emotion[key] = final_sorted_keys
# pretty print the top_10_for_each_emotion dictionary but only sho
print(json.dumps(top_10_for_each_emotion, indent=4, sort_keys=True))

    

mean total is 78.23181818181818
median total is 61
{
    "anger": [
        "Shakleton Shelter",
        "Seth Warner Shelter",
        "Toms Run Shelters",
        "Calf Mountain Shelter",
        "Thistle Hill Shelter",
        "Tom Leonard Shelter",
        "Mt. Wilcox South Shelters",
        "The Hemlocks Shelter",
        "Glen Brook Shelter",
        "Birch Run Shelter"
    ],
    "disgust": [
        "Wapiti Shelter",
        "Docs Knob Shelter",
        "Kirkridge Shelter",
        "David Lesser Memorial Shelter",
        "Peru Peak Shelter",
        "Little Rock Pond Shelter",
        "Wiley Shelter",
        "Blackrock Hut",
        "Kid Gore Shelter",
        "Story Spring Shelter"
    ],
    "fear": [
        "Leroy A. Smith Shelter",
        "Kirkridge Shelter",
        "Peters Mountain Shelter",
        "Spruce Peak Shelter",
        "Bromley Shelter",
        "Bake Oven Knob Shelter",
        "Brink Road Shelter",
        "Pickle Branch Shelter",
        "Jim & Molly De

In [34]:
data = []


for emotion, shelters in top_10_for_each_emotion.items():
    for rank, shelter in enumerate(shelters, start=1):
        lat = shelter_data[shelter]["cordinates"]["latitude"]
        lon = shelter_data[shelter]["cordinates"]["longitude"]
        data.append({"Emotion": emotion, "Rank": rank, "Shelter": shelter, "Latitude": lat, "Longitude": lon})


df_map = pd.DataFrame(data)
print(df_map.info)


<bound method DataFrame.info of      Emotion  Rank                        Shelter  Latitude  Longitude
0        joy     1    Tucker Johnson Camping Area  43.69112  -72.90010
1        joy     2         Double Springs Shelter  36.50897  -81.98620
2        joy     3         Bearfence Mountain Hut  38.44408  -78.47070
3        joy     4        Icewater Spring Shelter  35.62969  -83.38640
4        joy     5       Pochuck Mountain Shelter  41.27167  -74.51630
5        joy     6   Stewart Hollow Brook Shelter  41.77860  -73.41980
6        joy     7           Alec Kennedy Shelter  40.11192  -77.10430
7        joy     8           Clarks Ferry Shelter  40.39222  -76.99400
8        joy     9       Seeley-Woodworth Shelter  37.81893  -79.15510
9        joy    10         Fullhardt Knob Shelter  37.39847  -79.85390
10   sadness     1             Rice Field Shelter  37.37479  -80.75860
11   sadness     2                Bromley Shelter  43.23950  -72.93720
12   sadness     3         Gooch Mountain She

In [62]:
from shapely.geometry import Point, shape
import json

def get_state(lat, lon, geojson_path='../inputData/us-states.json'):
    # Load GeoJSON file
    with open(geojson_path) as f:
        geojson_data = json.load(f)
    print(len(geojson_data))
    point = Point(lon, lat)

    # Iterate through features in GeoJSON and check if the point is within any polygon
    for feature in geojson_data['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            return feature['properties']['NAME']

    # If the point is not within any state, return None or a default value
    return None

# Example usage
latitude = 37.7749
longitude = -122.4194
geojson_path = 'path/to/your/states.geojson'

state = get_state(latitude, longitude)

df.info(1)



2
<class 'pandas.core.frame.DataFrame'>
Index: 14436 entries, 5 to 33262
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date                14436 non-null  object 
 1   Hiker trail name    14436 non-null  object 
 2   Hiker Journal Link  14436 non-null  object 
 3   Journal Story       14436 non-null  object 
 4   Start location      14436 non-null  object 
 5   Destination         14430 non-null  object 
 6   Today Miles         14435 non-null  object 
 7   Latitude            14436 non-null  float64
 8   Longitude           14436 non-null  float64
 9   State               3300 non-null   object 
 10  Total Shelters      14436 non-null  object 
 11  Occurrence          14431 non-null  object 
 12  year                14436 non-null  int64  
 13  month               14436 non-null  int64  
 14  label               14436 non-null  object 
 15  Emotion_scores      14436 non-null  object 
 16  label_i

In [69]:
import folium
from folium.plugins import BeautifyIcon


# Create a folium map centered at a specific location
map_center = [41.90722, -70.0369]
folium_map = folium.Map(location=map_center, zoom_start=5, tiles="CartoDB positron")
currEmotion = "surprise"
title_html = f'<h3 align="center" style="font-size:16px"><b>Top 10 locations for emotion: {currEmotion}</b></h3>'   
folium_map.get_root().html.add_child(folium.Element(title_html))

# find the top 1 shelters for the currEmotion
top_1_shelter = top_10_for_each_emotion[currEmotion][0]
# add a legend to the map that shows the names of the top 3 shelters for each currEmotion
legend_html = f'''
        <div style="position: fixed;
                    bottom: 50px; right: 50px; width: 250px; height: 250px;
                    border:2px solid grey; z-index:9999; font-size:24px;
                    ">&nbsp; <b>Legend</b> <br>
                        &nbsp; 1. "{top_10_for_each_emotion[currEmotion][0]}, State: {get_state(shelter_data[top_10_for_each_emotion[currEmotion][0]]['cordinates']['latitude'], shelter_data[top_10_for_each_emotion[currEmotion][0]]['cordinates']['longitude'])} <br>
                        &nbsp; 2. {top_10_for_each_emotion[currEmotion][1]} <br>
                        &nbsp; 3. {top_10_for_each_emotion[currEmotion][2]} <br>
        </div>
        '''
folium_map.get_root().html.add_child(folium.Element(legend_html))


# Add arrow markers to the map
for index, row in df_map.iterrows():
    # Create a BeautifyIcon arrow marker

    if row["Emotion"] != currEmotion:
        continue
    

    icon = None
    
    if row["Rank"] == 1:
        icon = BeautifyIcon(
        icon_shape='circle',
        number=int(row["Rank"]),
        border_color=row["Emotion"],
        background_color='gold',
        inner_icon_style='color: black;',
        icon_size=[50, 50]
    )
    elif row["Rank"] == 2:
        icon = BeautifyIcon(
            icon_shape='circle',
            number=int(row["Rank"]),
            border_color=row["Emotion"],
            background_color='silver',
            inner_icon_style='color: black;',
            icon_size=[50, 50])
    elif row["Rank"] == 3:
        icon = BeautifyIcon(
            icon_shape='circle',
            number=int(row["Rank"]),
            border_color=row["Emotion"],
            background_color='brown',
            inner_icon_style='color: black;',
            icon_size=[50, 50])
    else:
        icon = BeautifyIcon(
            icon='arrow-down',
            icon_shape='triangle',
            border_color=row["Emotion"],
            background_color='transparent',
            inner_icon_style='color: white;',
            icon_size=[30, 30]
        )


    # make the title of the map itself "top 10 shelters for surprise"
    
    # find the amount of times that the emotion is in the radius of the shelter
    cou = count_of_emotion_dictionary[row["Emotion"]][row["Shelter"]]
    folium.TileLayer

    popup_content = f"<b>Shelter:</b> {row['Shelter']}<br><b>Emotion:</b> {row['Emotion']}<br><b>Rank:</b> {row['Rank']}<br><b>Percentage:</b> {cou}<br><b>Total Entries: {amount_of_entries_at_shelter[row['Shelter']]}</b>"
    popup = folium.Popup(popup_content, max_width=300)

    # Add the arrow marker to the map
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        tooltip=popup_content,
        icon=icon,
    ).add_to(folium_map)

# Display the map
folium_map


2


# Code for Fear Map

In [78]:
import folium
from folium.plugins import MarkerCluster, BeautifyIcon


# Create a folium map centered at a specific location
map_center = [41.90722, -70.0369]
fear_folium_map = folium.Map(location=map_center, zoom_start=5, tiles="CartoDB positron")
currEmotion = "fear"
title_html = f'<h3 align="center" style="font-size:16px"><b>Top 10 locations for emotion: {currEmotion}</b></h3>'   
fear_folium_map.get_root().html.add_child(folium.Element(title_html))


# add a legend to the map that shows the names of the top 3 shelters for each currEmotion
legend_html = f'''
        <div style="position: fixed;
                    bottom: 50px; right: 50px; width: 450px; height: 250px;
                    border:2px solid grey; z-index:9999; font-size:20px;
                    ">&nbsp; <b>Legend</b> <br>
                        &nbsp; 1. {top_10_for_each_emotion[currEmotion][0]}, {get_state(shelter_data[top_10_for_each_emotion[currEmotion][0]]['cordinates']['latitude'], shelter_data[top_10_for_each_emotion[currEmotion][0]]['cordinates']['longitude'])} <br>
                        &nbsp; 2. {top_10_for_each_emotion[currEmotion][1]}, {get_state(shelter_data[top_10_for_each_emotion[currEmotion][1]]['cordinates']['latitude'], shelter_data[top_10_for_each_emotion[currEmotion][1]]['cordinates']['longitude'])} <br>
                        &nbsp; 3. {top_10_for_each_emotion[currEmotion][2]}, {get_state(shelter_data[top_10_for_each_emotion[currEmotion][2]]['cordinates']['latitude'], shelter_data[top_10_for_each_emotion[currEmotion][2]]['cordinates']['longitude'])} <br>
        </div>
        '''
fear_folium_map.get_root().html.add_child(folium.Element(legend_html))


# Add arrow markers to the map
for index, row in df_map.iterrows():
    # Create a BeautifyIcon arrow marker

    if row["Emotion"] != currEmotion:
        continue
    

    icon = None
    
    if row["Rank"] == 1:
        icon = BeautifyIcon(
        border_color="#FFD700",
        text_color="#00ABDC",
        number=1,
    inner_icon_style="margin-top:0; font-size: 20px",
    icon_size=[40,40],

    )
    elif row["Rank"] == 2:
        icon = BeautifyIcon(
        border_color="#C0C0C0",
        text_color="#00ABDC",
        number=2,
    inner_icon_style="margin-top:0; font-size: 20px",
    icon_size=[41,41],
        )


    elif row["Rank"] == 3:
        icon = BeautifyIcon(
        border_color="#CD7F32",
        text_color="#00ABDC",
        number=3,
    inner_icon_style="margin-top:0; font-size: 20px",
    icon_size=[41,41],
        )
    else:
        icon = BeautifyIcon(
            icon = "house",
        border_color="#CD7F32",
        text_color="#00ABDC",
        
    inner_icon_style="margin-top:0;",)


    # make the title of the map itself "top 10 shelters for surprise"
    
    # find the amount of times that the emotion is in the radius of the shelter
    cou = count_of_emotion_dictionary[row["Emotion"]][row["Shelter"]]

    popup_content = f"<b>Shelter:</b> {row['Shelter']}<br><b>Emotion:</b> {row['Emotion']}<br><b>Rank:</b> {row['Rank']}<br><b>Percentage of Emotion in a 5 mile radius:</b> {cou}%<br><b>Total Entries: {amount_of_entries_at_shelter[row['Shelter']]}</b>"
    popup = folium.Popup(popup_content, max_width=300)

    # Add the arrow marker to the map
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        icon=icon,
        popup=popup
    ).add_to(fear_folium_map)

# Display the map
fear_folium_map


2
2
2


# Code for Sadness

In [80]:
import folium
from folium.plugins import MarkerCluster, BeautifyIcon


# Create a folium map centered at a specific location
map_center = [41.90722, -70.0369]
sad_folium_map = folium.Map(location=map_center, zoom_start=5, tiles="CartoDB positron")
currEmotion = "sadness"
title_html = f'<h3 align="center" style="font-size:16px"><b>Top 10 locations for emotion: {currEmotion}</b></h3>'   
sad_folium_map.get_root().html.add_child(folium.Element(title_html))


# add a legend to the map that shows the names of the top 3 shelters for each currEmotion
legend_html = f'''
        <div style="position: fixed;
                    bottom: 50px; right: 50px; width: 450px; height: 250px;
                    border:2px solid grey; z-index:9999; font-size:20px;
                    ">&nbsp; <b>Legend</b> <br>
                        &nbsp; 1. {top_10_for_each_emotion[currEmotion][0]}, {get_state(shelter_data[top_10_for_each_emotion[currEmotion][0]]['cordinates']['latitude'], shelter_data[top_10_for_each_emotion[currEmotion][0]]['cordinates']['longitude'])} <br>
                        &nbsp; 2. {top_10_for_each_emotion[currEmotion][1]}, {get_state(shelter_data[top_10_for_each_emotion[currEmotion][1]]['cordinates']['latitude'], shelter_data[top_10_for_each_emotion[currEmotion][1]]['cordinates']['longitude'])} <br>
                        &nbsp; 3. {top_10_for_each_emotion[currEmotion][2]}, {get_state(shelter_data[top_10_for_each_emotion[currEmotion][2]]['cordinates']['latitude'], shelter_data[top_10_for_each_emotion[currEmotion][2]]['cordinates']['longitude'])} <br>
        </div>
        '''
sad_folium_map.get_root().html.add_child(folium.Element(legend_html))


# Add arrow markers to the map
for index, row in df_map.iterrows():
    # Create a BeautifyIcon arrow marker

    if row["Emotion"] != currEmotion:
        continue
    

    icon = None
    
    if row["Rank"] == 1:
        icon = BeautifyIcon(
        border_color="#FFD700",
        text_color="#00ABDC",
        number=1,
    inner_icon_style="margin-top:0; font-size: 20px",
    icon_size=[40,40],

    )
    elif row["Rank"] == 2:
        icon = BeautifyIcon(
        border_color="#C0C0C0",
        text_color="#00ABDC",
        number=2,
    inner_icon_style="margin-top:0; font-size: 20px",
    icon_size=[41,41],
        )


    elif row["Rank"] == 3:
        icon = BeautifyIcon(
        border_color="#CD7F32",
        text_color="#00ABDC",
        number=3,
    inner_icon_style="margin-top:0; font-size: 20px",
    icon_size=[41,41],
        )
    else:
        icon = BeautifyIcon(
            icon = "house",
        border_color="#CD7F32",
        text_color="#00ABDC",
        
    inner_icon_style="margin-top:0;",)


    # make the title of the map itself "top 10 shelters for surprise"
    
    # find the amount of times that the emotion is in the radius of the shelter
    cou = count_of_emotion_dictionary[row["Emotion"]][row["Shelter"]]

    popup_content = f"<b>Shelter:</b> {row['Shelter']}<br><b>Emotion:</b> {row['Emotion']}<br><b>Rank:</b> {row['Rank']}<br><b>Percentage of Emotion in a 5 mile radius:</b> {cou}%<br><b>Total Entries: {amount_of_entries_at_shelter[row['Shelter']]}</b>"
    popup = folium.Popup(popup_content, max_width=300)

    # Add the arrow marker to the map
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        icon=icon,
        popup=popup
    ).add_to(sad_folium_map)

# Display the map
sad_folium_map


2
2
2


# Code for Joy

In [81]:
import folium
from folium.plugins import MarkerCluster, BeautifyIcon


# Create a folium map centered at a specific location
map_center = [41.90722, -70.0369]
joy_folium_map = folium.Map(location=map_center, zoom_start=5, tiles="CartoDB positron")
currEmotion = "joy"
title_html = f'<h3 align="center" style="font-size:16px"><b>Top 10 locations for emotion: {currEmotion}</b></h3>'   
joy_folium_map.get_root().html.add_child(folium.Element(title_html))


# add a legend to the map that shows the names of the top 3 shelters for each currEmotion
legend_html = f'''
        <div style="position: fixed;
                    bottom: 50px; right: 50px; width: 450px; height: 250px;
                    border:2px solid grey; z-index:9999; font-size:20px;
                    ">&nbsp; <b>Legend</b> <br>
                        &nbsp; 1. {top_10_for_each_emotion[currEmotion][0]}, {get_state(shelter_data[top_10_for_each_emotion[currEmotion][0]]['cordinates']['latitude'], shelter_data[top_10_for_each_emotion[currEmotion][0]]['cordinates']['longitude'])} <br>
                        &nbsp; 2. {top_10_for_each_emotion[currEmotion][1]}, {get_state(shelter_data[top_10_for_each_emotion[currEmotion][1]]['cordinates']['latitude'], shelter_data[top_10_for_each_emotion[currEmotion][1]]['cordinates']['longitude'])} <br>
                        &nbsp; 3. {top_10_for_each_emotion[currEmotion][2]}, {get_state(shelter_data[top_10_for_each_emotion[currEmotion][2]]['cordinates']['latitude'], shelter_data[top_10_for_each_emotion[currEmotion][2]]['cordinates']['longitude'])} <br>
        </div>
        '''
joy_folium_map.get_root().html.add_child(folium.Element(legend_html))


# Add arrow markers to the map
for index, row in df_map.iterrows():
    # Create a BeautifyIcon arrow marker

    if row["Emotion"] != currEmotion:
        continue
    

    icon = None
    
    if row["Rank"] == 1:
        icon = BeautifyIcon(
        border_color="#FFD700",
        text_color="#00ABDC",
        number=1,
    inner_icon_style="margin-top:0; font-size: 20px",
    icon_size=[40,40],

    )
    elif row["Rank"] == 2:
        icon = BeautifyIcon(
        border_color="#C0C0C0",
        text_color="#00ABDC",
        number=2,
    inner_icon_style="margin-top:0; font-size: 20px",
    icon_size=[41,41],
        )


    elif row["Rank"] == 3:
        icon = BeautifyIcon(
        border_color="#CD7F32",
        text_color="#00ABDC",
        number=3,
    inner_icon_style="margin-top:0; font-size: 20px",
    icon_size=[41,41],
        )
    else:
        icon = BeautifyIcon(
            icon = "house",
        border_color="#CD7F32",
        text_color="#00ABDC",
        
    inner_icon_style="margin-top:0;",)


    # make the title of the map itself "top 10 shelters for surprise"
    
    # find the amount of times that the emotion is in the radius of the shelter
    cou = count_of_emotion_dictionary[row["Emotion"]][row["Shelter"]]

    popup_content = f"<b>Shelter:</b> {row['Shelter']}<br><b>Emotion:</b> {row['Emotion']}<br><b>Rank:</b> {row['Rank']}<br><b>Percentage of Emotion in a 5 mile radius:</b> {cou}%<br><b>Total Entries: {amount_of_entries_at_shelter[row['Shelter']]}</b>"
    popup = folium.Popup(popup_content, max_width=300)

    # Add the arrow marker to the map
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        icon=icon,
        popup=popup
    ).add_to(joy_folium_map)

# Display the map
joy_folium_map


2
2
2
